In [1]:
import os
import sys

os.environ['CUDA_VISIBLE_DEVICES'] = '3'
sys.path.append('../')

import pickle
import wandb
import torch
import torch.nn as nn

from omegaconf import OmegaConf
from tqdm.auto import tqdm
from model.unet import get_unet
from data_utils import get_pmri_data_loaders, get_eval_data
from trainer.unet_trainer import get_unet_trainer
from adapters import PCA_Adapter, PCAModuleWrapper
from torch.utils.data import DataLoader

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Load basic config
DATA_KEY = 'prostate'
ITERATION = 1
LOG = False
AUGMENT = False
LOAD_ONLY_PRESENT = True
SUBSET = 'training' # Whether the validation is a subset or the whole set, normally bool, but for eval must be string 'training'
VALIDATION = True # If false makes validation set be the training one
N_DIMS = [2, 4, 8, 16, 32, 64]
cfg = OmegaConf.load('../configs/conf.yaml')
OmegaConf.update(cfg, 'run.iteration', ITERATION)
OmegaConf.update(cfg, 'run.data_key', DATA_KEY)

unet_name = 'monai-64-4-4'
extra_description = 'moredata'
cfg.wandb.project = f'{DATA_KEY}_{unet_name}_{ITERATION}{extra_description}'
args = unet_name.split('-')
cfg.unet[DATA_KEY].pre = unet_name
cfg.unet[DATA_KEY].arch = args[0]
cfg.unet[DATA_KEY].n_filters_init = None if unet_name == 'swinunetr' else int(args[1])
cfg.unet[DATA_KEY].training.augment = AUGMENT
cfg.unet[DATA_KEY].training.validation = VALIDATION
cfg.unet[DATA_KEY].training.subset = SUBSET
cfg.unet[DATA_KEY].training.load_only_present = LOAD_ONLY_PRESENT
cfg.unet[DATA_KEY].training.batch_size = 58

if args[0] == 'monai':
    cfg.unet[DATA_KEY].depth = int(args[2])
    cfg.unet[DATA_KEY].num_res_units = int(args[3])

if LOG:
    wandb.init(
        project=cfg.wandb.project,
        config={
            "learning_rate": cfg.unet[DATA_KEY].training.lr,
            "architecture": unet_name,
            "dataset": DATA_KEY
        }
    )

layer_names = ['model.0.conv',
               'model.1.submodule.0.conv',
               'model.1.submodule.1.submodule.0.conv',
               'model.1.submodule.1.submodule.1.submodule.0.conv']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
if AUGMENT:
    cfg.format = 'torch'
    train_loader, val_loader = get_pmri_data_loaders(cfg=cfg)
else:
    cfg.format = 'numpy'
    data = get_eval_data(train_set=False, val_set=False, eval_set=True, cfg=cfg)
    dataset = data['eval']
    print(f'Length of non-aug dataset: {len(dataset)}')

Loading evaluation PMRI dataset for vendor siemens ...
Length of non-aug dataset: 986


In [4]:
try:
    for n_dims in N_DIMS[::-1]:
        adapters = [PCA_Adapter(swivel, n_dims, cfg.unet[DATA_KEY].training.batch_size,
                                pre_fit=True, train_gaussian=False, compute_dist=True,
                                name=cfg.wandb.project) for swivel in layer_names]
        adapters = nn.ModuleList(adapters)
        unet, state_dict = get_unet(cfg, update_cfg_with_swivels=False, return_state_dict=True)
        unet_adapted = PCAModuleWrapper(model=unet, adapters=adapters)
        unet_adapted.hook_adapters()
        unet_adapted.to(device);

        if AUGMENT:
            pmri_trainer = get_unet_trainer(cfg=cfg, train_loader=train_loader, val_loader=val_loader, model=unet_adapted)
            # TODO
        else:
            unet_adapted.eval()
            dataloader = DataLoader(dataset,
                                    batch_size=cfg.unet[DATA_KEY].training.batch_size,
                                    shuffle=False, drop_last=False)

            for i, batch in enumerate(tqdm(dataloader)):
                input_ = batch['input'].to(device)
                unet_adapted(input_)
        break

finally:
    if LOG:
        wandb.finish()

Loaded IPCA from path/workspace/src/out/pca/prostate_monai-64-4-4_1moredata_model_0_conv_64dim.pkl
Loaded activations from path /workspace/src/out/activations/prostate_monai-64-4-4_1moredata_model_0_conv_activations_64dims.npy
Mean and inverse covariance matrix computed and set
Emptied collected activations of adapter
Loaded IPCA from path/workspace/src/out/pca/prostate_monai-64-4-4_1moredata_model_1_submodule_0_conv_64dim.pkl
Loaded activations from path /workspace/src/out/activations/prostate_monai-64-4-4_1moredata_model_1_submodule_0_conv_activations_64dims.npy
Mean and inverse covariance matrix computed and set
Emptied collected activations of adapter
Loaded IPCA from path/workspace/src/out/pca/prostate_monai-64-4-4_1moredata_model_1_submodule_1_submodule_0_conv_64dim.pkl
Loaded activations from path /workspace/src/out/activations/prostate_monai-64-4-4_1moredata_model_1_submodule_1_submodule_0_conv_activations_64dims.npy
Mean and inverse covariance matrix computed and set
Emptied c

  0%|          | 0/17 [00:00<?, ?it/s]

torch.Size([58, 64]) torch.Size([1, 64]) torch.Size([1, 64, 64])


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3

In [7]:
lname = layer_names[2].replace('.', '_')

if AUGMENT: lname += '_aug'

for n_dims in N_DIMS:
    print(f'Loading IPCA of {n_dims} dims')
    with open(f'/workspace/src/out/pca/{cfg.wandb.project}_{lname}_{n_dims}dim.pkl',  'rb') as f:
        ipca = pickle.load(f)
    evr = ipca.explained_variance_ratio_
    cum_evr = np.cumsum(evr)
    # num_comp = np.argmax(cum_evr >= 0.95) + 1
    print(f'Max retained variance with all components: {cum_evr[-1]}')
    print(f'Loading activation maps obtained with IPCA of {n_dims} dims')
    act = np.load(f'/workspace/src/out/activations/{cfg.wandb.project}_{lname}_activations_{n_dims}dims.npy')
    print(f'Shape of activations: {act.shape}')

Loading IPCA of 2 dims
Max retained variance with all components: 0.09506224336175023
Loading activation maps obtained with IPCA of 2 dims
Shape of activations: (986, 2)
Loading IPCA of 4 dims
Max retained variance with all components: 0.12306879410929314
Loading activation maps obtained with IPCA of 4 dims
Shape of activations: (986, 4)
Loading IPCA of 8 dims
Max retained variance with all components: 0.15757514849147605
Loading activation maps obtained with IPCA of 8 dims
Shape of activations: (986, 8)
Loading IPCA of 16 dims
Max retained variance with all components: 0.2094781082001532
Loading activation maps obtained with IPCA of 16 dims
Shape of activations: (986, 16)
Loading IPCA of 32 dims
Max retained variance with all components: 0.2809973327741017
Loading activation maps obtained with IPCA of 32 dims
Shape of activations: (986, 32)
Loading IPCA of 64 dims
Max retained variance with all components: 0.38633788359479576
Loading activation maps obtained with IPCA of 64 dims
Shape